# `TODO`  

``` HTML
[ANALYSE]   >> Remove vertices with low confidence (using 'R2' values from the PRF results)
[ANALYSE]   >> Average analysis results across subjects

[ANALYSE]   >> Distance calculations per grid -- <DONE>
[ANALYSE]   >> Divide metric into grids and compute distances with variance of groups -- <DONE>
[ANALYSE]   >> Find way to compute 'correlation' between space-metric space plots -- <DONE> (TSNE-PLOTS!)
[VISUALIZE] >> TSNE embedding in space-metric plot -- <DONE>
[VISUALIZE] >> Polar plots per functional pracellation -- <DONE>
[VISUALIZE] >> Create polar heatmap of distances from distance analysis on grids -- <DONE>
```

---

# Imports

In [1]:
import sys
import os

In [2]:
import math

In [3]:
from itertools import chain, cycle

---

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [7]:
import seaborn as sns

In [8]:
import scipy as sc

In [9]:
from sklearn.manifold import TSNE

---

In [10]:
import mat73

---

In [11]:
import neuropythy as ny

In [12]:
import nibabel as nib

In [13]:
# import hcp_utils as hcp

---

---

In [14]:
import logging

In [15]:
import warnings

In [16]:
from IPython.display import display, display_html

In [17]:
from tqdm import tqdm

---

In [18]:
from utils import HCP_S3

## Config

In [19]:
# logging
logging.getLogger().setLevel(logging.INFO)

In [20]:
# warnings
# warnings.filterwarnings('ignore')

# ---
# Constants

In [21]:
ANG_MESH = np.linspace(0, 360, 61)
ECC_MESH = np.logspace(0, 4, 9, base=2) - 1 # np.array([0., 2, 5, 8, 12, 16]) # 

# Functions

In [22]:
def get_coors(sid=100610, volume_space='native', surface_mesh='fs_lr_low', surface_type='midthickness', msm_alignment_method='msmall', hemisphere='both', directory='hcp-subjects'):
    """ Gets the grayordinate coordinates of a subject for the given volume and surface space, surface type, surface alignment method, and hemisphere. 

    Example of files to get,
    - 100610/T1w/fsaverage_LR32k/100610.L.midthickness_MSMAll.32k_fs_LR.surf.gii
    - 100610/T1w/fsaverage_LR32k/100610.R.midthickness_MSMAll.32k_fs_LR.surf.gii
    - 100610/MNINonLinear/fsaverage_LR32k/100610.L.atlasroi.32k_fs_LR.shape.gii
    - 100610/MNINonLinear/fsaverage_LR32k/100610.R.atlasroi.32k_fs_LR.shape.gii

    Parameters
    ----------
    sid : int, optional
        Serial ID of the participant, by default 100610.
    volume_space : {'native', 'mni'}, optional
        The volume space for the coordinates, by default 'native'.
    surface_mesh : {'native', 'fs_lr_high', 'fs_lr_med', 'fs_lr_low'}, optional
        The surface mesh for the coordinates, by default 'native'.
    surface_type : {flat, white, pial, midthickness, sphere, inflated, very_inflated}, optional
        The surface type for the coordinates, by default 'midthickness'.
    msm_alignment_method: {'msmsulc', 'msmall', None}, optional
        The surface alignment method used by MSM, by default 'msmall'.
    hemisphere : {'left', 'right', 'both'}, optional
        The hemisphere to get coordinates for, by default 'both'.
    directory : str, optional
        The path to HCP subjects root directory under './data/', by default 'hcp-subjects'.

    Returns
    -------
    numpy.darray
        List of 3D coordinates of the requested surface; [left_hemisphere_vertices; right_hemisphere_cortices]. # ; subcortical_voxels.
    """
    
    if msm_alignment_method is None and not surface_mesh == 'native' and surface_type not in {'flat', 'sphere'}:
        logging.warning('`None` as MSM alignment method is only valid for native surface mesh (probably also only in MNI volume space)! Defaulting to MSMAll method.')
        msm_alignment_method = 'msmall'
    
    if surface_type != 'sphere' and volume_space == 'mni' and surface_mesh == 'native':
        raise ValueError('Only spherical surface type available for native surface mesh in MNI volume space!')
    
    if volume_space == 'native' and surface_mesh == 'fs_lr_high':
        raise ValueError('164k vertex high resolution Conte69 registered standard surface mesh only available in MNI volume space!')
    
    if volume_space == 'native' and surface_mesh == 'native':
        if msm_alignment_method == 'msmall':
            logging.warning('MSMAll alignment method not available for native surface mesh in native volume space! Using MSMSulc method instead.')
        logging.info('The method is not specified in the available files for native surface mesh in native volume space. Probably MSMSulc.')
        msm_alignment_method = None
    
    volume_space_label = {'native': 'T1w', 'mni': 'MNINonLinear'}[volume_space]
    surface_mesh_label = {'native': 'Native', 'fs_lr_high': '.', 'fs_lr_med': 'fsaverage_LR59k', 'fs_lr_low': 'fsaverage_LR32k'}[surface_mesh]
    surface_mesh_label_2 = {'native': 'native', 'fs_lr_high': '164k_fs_LR', 'fs_lr_med': '59k_fs_LR', 'fs_lr_low': '32k_fs_LR'}[surface_mesh]
    # msm_alignment_method_label = 
    if surface_type in {'flat', 'sphere'}:
        if msm_alignment_method == 'msmall':
            logging.warning(f'MSMAll alignment method not available for {surface_type} surface mesh! Using MSMSulc method instead.')
        logging.info('The method is not specified in the available files for flat and spherical meshes. Probably MSMSulc.')
        msm_alignment_method_label = ''
    elif surface_mesh == 'native':
        msm_alignment_method_label = {'msmsulc': '.MSMAll', 'msmall': '.MSMSulc', None: ''}[msm_alignment_method]
    elif surface_mesh == 'fs_lr_med':
        msm_alignment_method_label = {'msmsulc': '', 'msmall': '_1.6mm_MSMAll'}[msm_alignment_method]
    else:
        msm_alignment_method_label = {'msmsulc': '', 'msmall': '_MSMAll'}[msm_alignment_method]
    
    if hemisphere == 'both':
        lh_coors_file = f'{sid}/{volume_space_label}/{surface_mesh_label}/{sid}.L.{surface_type}{msm_alignment_method_label}.{surface_mesh_label_2}.surf.gii'
        # print(lh_coors_file)
        if not os.path.isfile(f'data/{directory}/{lh_coors_file}'):
            HCP_S3.download(f'HCP_1200/{lh_coors_file}', local=f'data/{directory}', trim=2)
        lh_coors = nib.load(f'data/{directory}/{lh_coors_file}').agg_data()[0]
        
        rh_coors_file = f'{sid}/{volume_space_label}/{surface_mesh_label}/{sid}.R.{surface_type}{msm_alignment_method_label}.{surface_mesh_label_2}.surf.gii'
        # print(rh_coors_file)
        if not os.path.isfile(f'data/{directory}/{rh_coors_file}'):
            HCP_S3.download(f'HCP_1200/{rh_coors_file}', local=f'data/{directory}', trim=2)
        rh_coors = nib.load(f'data/{directory}/{rh_coors_file}').agg_data()[0]
        
        
        lh_atlas_file = f'{sid}/MNINonLinear/{surface_mesh_label}/{sid}.L.atlasroi.{surface_mesh_label_2}.shape.gii'
        # print(lh_atlas_file)
        if not os.path.isfile(f'data/{directory}/{lh_atlas_file}'):
            HCP_S3.download(f'HCP_1200/{lh_atlas_file}', local=f'data/{directory}', trim=2)
        lh_atlas = nib.load(f'data/{directory}/{lh_atlas_file}').agg_data().astype(bool)
        
        rh_atlas_file = f'{sid}/MNINonLinear/{surface_mesh_label}/{sid}.R.atlasroi.{surface_mesh_label_2}.shape.gii'
        # print(rh_atlas_file)
        if not os.path.isfile(f'data/{directory}/{rh_atlas_file}'):
            HCP_S3.download(f'HCP_1200/{rh_atlas_file}', local=f'data/{directory}', trim=2)
        rh_atlas = nib.load(f'data/{directory}/{rh_atlas_file}').agg_data().astype(bool)
        
        return np.r_[lh_coors[lh_atlas], rh_coors[rh_atlas]], ['L'] * lh_atlas.sum() + ['R'] * rh_atlas.sum()
    
    else:
        hemisphere_label = {'left': 'L', 'right': 'R'}[hemisphere]
        
        h_coors_file = f'{sid}/{volume_space_label}/{surface_mesh_label}/{sid}.{hemisphere_label}.{surface_type}{msm_alignment_method_label}.{surface_mesh_label_2}.surf.gii'
        # print(h_coors_file)
        if not os.path.isfile(f'data/{directory}/{h_coors_file}'):
            HCP_S3.download(f'HCP_1200/{h_coors_file}', local=f'data/{directory}', trim=2)
        h_coors = nib.load(f'data/{directory}/{h_coors_file}').agg_data()[0]

        h_atlas_file = f'{sid}/MNINonLinear/{surface_mesh_label}/{sid}.{hemisphere_label}.atlasroi.{surface_mesh_label_2}.shape.gii'
        # print(h_atlas_file)
        if not os.path.isfile(f'data/{directory}/{h_atlas_file}'):
            HCP_S3.download(f'HCP_1200/{h_atlas_file}', local=f'data/{directory}', trim=2)
        h_atlas = nib.load(f'data/{directory}/{h_atlas_file}').agg_data().astype(bool)
        
        return h_coors[h_atlas], [hemisphere_label] * h_atlas.sum()

> `len(lh_coors) + len(rh_coors) + sum(lh_atlas) + sum(rh_atlas) - len(df)`  
>> $33114 = 32492 * 2 + 29696 + 29716 - 91282$

---

In [23]:
rectify = relu = lambda x, elbow=0.0, norm=1.0: (type(x) if type(x) is not np.ndarray else np.array)(np.c_[x, np.zeros_like(x)].max(axis=1)/norm)

---

In [24]:
def group(df_, ang_mesh=ANG_MESH, ecc_mesh=ECC_MESH, aggr_func='mean'): # spatial_grid_spacing in mm
    """ Groups, and aggregates dataframe in passed eccentricity and angular ranges using passed aggregation function, by parcel.

    Parameters
    ----------
    df_ : pandas.DataFrame
        The dataframe to be consolidated.
    ang_mesh : numpy.ndarray, optional
        The angular mesh to cut `df_['ang']` with, by default `ANG_MESH`.
    ecc_mesh : numpy.ndarray, optional
        The eccentricity mesh to cut `df_['ecc']` with, by default `ECC_MESH`.
    aggr_func : function or str, optional
        The aggregation function used to aggregate groups (vertices in mesh polygons), by default 'mean'.
        
    Returns
    -------
    pandas.DataFrame
        Dataframe agregated in eccentricty and angular ranges, grouped by parcels.
    """
    def wrapper(df_grp):
        df_grp_ = df_grp.groupby([
            pd.cut(df_grp['ecc'], ecc_mesh, labels=ecc_mesh[1:].round(1)),
            pd.cut(df_grp['ang'], ang_mesh, labels=((ang_mesh[1:] + ang_mesh[:-1])/2).astype(int))
        ])
        if type(aggr_func) == str: # 'mean'
            return getattr(df_grp_, aggr_func)(numeric_only=True)
        elif callable(aggr_func): # lambda x: x.mean()
            return df_grp_.apply(aggr_func) # lambda x: aggr_func(x[x.notnull()])
    
    return df_.groupby(label).apply(wrapper)

In [25]:
def l2(df_, roi_, shift_len=None):
    """ Computes distances between parcels.

    Parameters
    ----------
    df_ : pandas.DataFrame, optional
        DataFrame grouped by parcels and eccentricity/angles.
    roi_ : array-like
        Parcels in the passed dataframe.
    shift_len : int or `None`, optional
        The number of groups available for each parcel, by default `None`.
        Is calculated automatically (`len(df_.loc['V1'])`) if `None`.
        
    Returns
    -------
    pandas.DataFrame
        Dataframe with canonical directional distances ['dx', 'dy', 'dz'] and L2 distance between vertices (voxels) ['d'].
    """
    shift_len = shift_len if shift_len is not None else len(df_.loc['V1'])
    diff = lambda x: (x - x.shift(-shift_len)).add_prefix('d')
    
    df_d = diff(df_) # [['dx', 'dy', 'dz']]
        
    df_d['d'] = np.sqrt(np.sum(df_d.pow(2), axis=1))
    # Improv.: https://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians
    
    df_d['d'][df_d.isna().any(axis=1)] = np.nan
    
    df_d = df_d.iloc[:-shift_len]
    df_d.index = df_d.index.set_levels([f'{i} - {j}' for i, j in zip(roi_[:-1], roi_[1:])], level=0)
    
    return df_d # [['dx', 'dy', 'dz'] + ['d']]

---

In [26]:
def display_side_by_side(*args, titles=cycle([''])):
    """ Display Dataframes side by side """
    display_html(''.join([fr"""
            <th style="text-align:center">
                <td style="vertical-align:top">
                    <h2 style="text-align: center;">{title}</h2>
                    {df_.to_html().replace('table', 'table style="display:inline"')}
                </td>
            </th>
        """ for df_, title in zip(args, chain(titles, cycle(['</br>'])))]), raw=True)

# Load Data

In [27]:
data = mat73.loadmat('data/osfstorage-archive/prfresults.mat')

In [28]:
data.keys()

dict_keys(['allresults', 'ciftifsaveragebad', 'ciftifsaverageix', 'groupsubjectids', 'quants', 'subjectids'])

- `allresults` - is 91282 grayordinates x 6 quantities x 184 datasets x 3 model fits with the full set of pRF analysis results.  
- `quants` - is a 1 x 6 cell vector with a label for the 6 quantities: {'ang' 'ecc' 'gain' 'meanvol' 'R2' 'rfsize'}.  
- `subjectids` - is 184 x 1 with the 6-digit ID for each subject (both individual and group-average subjects).  
- `ciftifsaverageix` - is 327684 x 1 with indices into the 91282 CIFTI space. This indexing vector performs a nearest-neighbor mapping from CIFTI to FreeSurfer's fsaverage space. Note that some of the fsaverage vertices do not have a counterpart in the CIFTI space; these vertices are assigned a value of 1 in 'ciftifsaverageix'.  
- `ciftifsaveragebad` - is 327684 x 1 with logical values indicating which fsaverage vertices do not have a counterpart in the CIFTI space. These vertices should receive a NaN after performing the indexing-based mapping.  
- `groupsubjectids` - is a 1 x 3 cell vector indicating which subjects contributed to the special 999997, 999998, and 999999 group-average subjects, respectively. Each element is a vector of indices (in the range 1-181) in sorted order.

In [29]:
data["allresults"].shape

(91282, 6, 184, 3)

91282 grayordinates x 6 quantities x 184 datasets x 3 model fits

In [30]:
data["quants"]

['ang', 'ecc', 'gain', 'meanvol', 'R2', 'rfsize']

- `ang` is the angle of the estimated pRF location. Specifically, it is the angle that the center of the pRF has with respect to the positive x-axis. The range of values is 0-360 and the units are degrees. Note that when the estimated pRF is exactly at the center of gaze (i.e., eccentricity is exactly 0), then angle is ill-defined. Thus, we have explicitly set the angle to NaN for any case in which eccentricity is 0.  
- `ecc` is the eccentricity of the estimated pRF location. Specifically, it is the distance from the center of gaze to the center of the pRF. Values are greater than or equal to 0 and the units are degrees of visual angle.  
- `gain` is the BOLD response amplitude estimated in the model fit. The units are raw scanner units (same units as the time-series data in the CIFTI files). The value can be interpreted as the amplitude of the BOLD response that is predicted to result from 1 s of full-field visual stimulation.  
- `meanvol` is the mean signal intensity observed in the time-series data. The units are raw scanner units (same units as the time-series data in the CIFTI files). Note that some grayordinates have a mean signal intensity that are zero or negative, so be careful (e.g., if converting to percent BOLD signal change).  
- `R2` is the amount of variance (R-squared) in the time-series data explained by the pRF model. The values generally range from 0-100 and the units are percentages. Note that R-squared values are computed after projecting out low-order polynomials from both the time-series data and the model fit. Because of this step, R-squared values can sometimes drop below 0%.  
- `rfsize` is the size of the estimated pRF. Specifically, it is one standard deviation of a 2D Gaussian that describes the behavior of the grayordinate (see paper for details). Values are positive and the units are degrees of visual angle.

In [31]:
# Load standard atlases
atlases = mat73.loadmat('data/osfstorage-archive/atlas.mat')

In [32]:
print(atlases['wang2015labels'])

['Unknown', 'V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 'IPS3', 'IPS4', 'IPS5', 'SPL1', 'FEF']


In [33]:
label ='parcellation_label'

### Subjects \<ALL\>

In [63]:
subjects = set(data['subjectids'].astype(int)) - {999997, 999998, 999999}

In [76]:
df = pd.DataFrame(np.vstack(np.moveaxis(data['allresults'][:, :, [s in subjects for s in data['subjectids']], 0], -1, 0)), columns=data['quants'])
df['sid'] = np.repeat(list(subjects), (n_vertices:=data['allresults'].shape[0]))

In [77]:
df[label] = np.tile(np.array(atlases['wang2015labels'])[atlases['wang2015'].astype(int)], (n_subjects:=len(subjects)))

In [78]:
df[['x', 'y', 'z']] = np.nan
df['hemisphere'] = None
for sid in (pbar:=tqdm(subjects)):
    coors, hemisphere = get_coors(sid)
    df.loc[df['sid'] == sid, ['x', 'y', 'z']].iloc[:len(coors)] = coors
    df.loc[df['sid'] == sid].iloc[:len(coors)]['hemisphere'] = hemisphere
    pbar.set_description(f'> {sid}')

> 198653: 100%|███████████████████████████████████████████████████████████████| 181/181 [00:11<00:00, 15.47it/s]


### Subject 100610

---

`Neuropythy subject object`

---

In [79]:
df_100610 = df.loc[df['sid'] == 100610]

---
# Main

## Data Setup

In [ ]:
roi_1 = ['V1d', 'V1v', 'V2d', 'V2v']
roi_2 = ['V1', 'V2']

In [ ]:
roi_colors_1 = dict(zip(roi_1, sns.color_palette(n_colors=len(roi_1))))
roi_colors_2 = dict(zip(roi_2, sns.color_palette(n_colors=len(roi_2))))

# roi_markers_1 = dict(zip(roi_1, ['*', 'X', 'o', None][:len(roi_1)]))
# roi_markers_2 = dict(zip(roi_2, ['*', 'X', 'o', None][:len(roi_2)])) 
# ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')
# ('*', '+', '-', '.', '/', 'O', 'X', '\', 'o', 'x', '|')

In [ ]:
dfv_100610_1 = df_100610[df[label].isin(roi_1)]
dfv_100610_1 = dfv_1.sort_values(by=label, key=lambda col: col.str.lower())

In [ ]:
dfv_100610_1.head()

In [ ]:
dfv_100610_2 = df_100610[df_100610[label].str.contains('|'.join(roi_2))]
dfv_100610_2 = dfv_100610_2.sort_values(by=label, key=lambda col: col.str.lower())
dfv_100610_2.loc[:, label] = dfv_100610_2.loc[:, label].str.replace(r'(V\d)\w', lambda s: s.group(1), regex=True)

In [ ]:
dfv_100610_2.head()

In [ ]:
def filter_labels(df_, roi=roi, merge_dorsal_ventral=False):
    """
    """
    df__ = df_[df_[label].str.contains('|'.join(roi))]
    df__ = df__.sort_values(by=label, key=lambda col: col.str.lower())
    if merge_dorsal_ventral:
        dfv__.loc[:, label].str.replace(r'(V\d)\w', lambda s: s.group(1), regex=True)
    return df__

---

In [ ]:
CASE = 2
# 2 - No subdivisions into dorsal and ventral sections, just 'V1' and 'V2'

In [ ]:
if CASE == 1:
    dfv_100610_ = dfv_1
elif CASE == 2:
    dfv = dfv_2
    roi = roi_2
    roi_colors = roi_colors_2
    # roi_markers = roi_markers_2

In [ ]:
if CASE == 1:
    dfv_100610 = dfv_1
    roi = roi_1
    roi_colors = roi_colors_1
    # roi_markers = roi_markers_1
elif CASE == 2:
    dfv = dfv_2
    roi = roi_2
    roi_colors = roi_colors_2
    # roi_markers = roi_markers_2

In [ ]:
if CASE == 1:
    dfv_100610 = dfv_1
    roi = roi_1
    roi_colors = roi_colors_1
    # roi_markers = roi_markers_1
elif CASE == 2:
    dfv = dfv_2
    roi = roi_2
    roi_colors = roi_colors_2
    # roi_markers = roi_markers_2

---

In [ ]:
dfv_r = dfv[dfv['hemisphere'] == 'R']
dfv_l = dfv[dfv['hemisphere'] == 'L']

## Basic Visualisation

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(len(roi), 2, figsize=(18, 2*len(roi)), sharex=True, sharey=True)

for i, label_i in enumerate(roi):
    for j, hemisphere in enumerate(['L', 'R']):
        for coordinate_axis in ['x', 'y', 'z']:
            axs[i][j].hist(dfv[(dfv[label].str.contains(label_i)) & (dfv['hemisphere'] == hemisphere)][coordinate_axis],
                       bins=20, edgecolor='black', alpha=0.6,
                       label=f'{f"{coordinate_axis.upper()} Coordinate in Talairach Space" if i == j == 0 else ""}'
            )
            axs[i][j].set_title(f'{label_i} | {hemisphere}')

axs[0][0].set_xlim([None, 112])

fig.supxlabel('Greyordinate Coordinate')
fig.suptitle('Histogram of Greyordinate Coordinates of V1 Vertices')
fig.legend()

plt.tight_layout()

---

In [ ]:
metric = 'ecc'
metric_limit = 3

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d'})

axs[0].scatter(*dfv[['x', 'y', 'z']].T.to_numpy(), s=10, c=[roi_colors[pl] for pl in dfv[label]])
axs[0].legend(handles=[mpatches.Patch(color=roi_color, label=roi_label) for roi_label, roi_color in roi_colors.items()])

for roi_label in roi:
    axs[1].scatter(*dfv[dfv[label] == roi_label][['x', 'y', 'z']].T.to_numpy(),
               s=10, #marker=roi_markers_[roi_label],
               c=[np.array(roi_colors[roi_label]) * relu(1 - met/metric_limit) for met in dfv[dfv[label] == roi_label][metric]]
    )
axs[1].legend(handles=[mpatches.Patch(color=roi_color, label=roi_label) for roi_label, roi_color in roi_colors.items()])
axs[1].set_title(fr'Color Gradient $\propto 1 - \frac{{{metric}}}{{{metric_limit}}}$, with ${metric} > {metric_limit}$ blackened')

fig.suptitle('Locations of Verticies (Voxels)')
plt.tight_layout()

---

In [ ]:
dfv_ = dfv.groupby(['hemisphere',
                    pd.cut(dfv['ecc'], ECC_MESH, labels=ECC_MESH[1:].round(1))
                   ]).count()

plt.figure(figsize=(20, 2))
sns.heatmap([dfv_[label].loc[s] for s in ('L', 'R')], annot=True, fmt='g')
plt.yticks((0, 1), ('L', 'R'))
plt.xticks(list(range(len(ECC_MESH)))[::1], ECC_MESH.round(1)[::1])

plt.title('Counts of Vertices in Eccentricity Bins of the Provided Mesh')
plt.tight_layout()

In [ ]:
dfv_ = dfv.groupby(['hemisphere',
                    pd.cut(dfv['ang'], ANG_MESH, labels=((ANG_MESH[1:] + ANG_MESH[:-1])/2).astype(int))
                   ]).count()

plt.figure(figsize=(20, 2))
sns.heatmap([dfv_[label].loc[s] for s in ('L', 'R')])
plt.yticks((0, 1), ('L', 'R'))
plt.xticks(list(range(len(ANG_MESH)))[::3], ANG_MESH.astype(int)[::3])

plt.title('Counts of Vertices in Angular Bins of the Provided Mesh')
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': 'polar'})

axs[0].scatter(dfv_l['ang'], np.log(1 + dfv_l['ecc']), s=5, c=[roi_colors[l] for l in dfv_l[label]], alpha=0.6)
axs[0].legend(handles=[mpatches.Patch(color=roi_color, label=roi_label) for roi_label, roi_color in roi_colors.items()])
axs[0].set_rgrids(np.log(1 + ECC_MESH), ECC_MESH.round(1), color='b', fontsize=6)
axs[0].set_title('L')

axs[1].scatter(dfv_r['ang'], np.log(1 + dfv_r['ecc']), s=5, c=[roi_colors[l] for l in dfv_r[label]], alpha=0.6)
axs[1].legend(handles=[mpatches.Patch(color=roi_color, label=roi_label) for roi_label, roi_color in roi_colors.items()])
axs[1].set_rgrids(np.log(1 + ECC_MESH), ECC_MESH.round(1), color='b', fontsize=6)
axs[1].set_title('R')

# ax.set_thetagrids(ANG_MESH[:-1], [fr'${ANG_MESH[i]}^{{\circ}}$' if i%3 == 0 else None for i in range(len(ANG_MESH[:-1]))])
# ax.set_thetagrids([0])

fig.suptitle('RF Locations in Log Scale')
plt.tight_layout()

In [ ]:
plt.scatter(*TSNE(n_components=2, init='pca', learning_rate='auto', perplexity=10).fit_transform(dfv[['x', 'y', 'z', 'ecc', 'ang']]).T, # , 'rfsize'
            c=np.array([roi_colors[pl] for pl in dfv[label]]) * relu(1 - dfv['ecc']/3)[:, None]);

## Analysis

In [ ]:
dfv_l_p  = group(dfv_l)[['x', 'y', 'z']]
dfv_l_d  = l2(dfv_l_p, roi)
dfv_l_d_ = dfv_l_d.reset_index().pivot(columns='ang', index='ecc', values='d')

In [ ]:
dfv_r_p  = group(dfv_r)[['x', 'y', 'z']]
dfv_r_d  = l2(dfv_r_p, roi)
dfv_r_d_ = dfv_r_d.reset_index().pivot(columns='ang', index='ecc', values='d')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': 'polar'})

# sns.heatmap(dfv_l_d, cmap=sns.cm.rocket_r, ax=axs[0])
pplot_1 = axs[0].pcolormesh(ANG_MESH, np.log(1 + ECC_MESH), dfv_l_d_.to_numpy(), cmap=sns.cm.rocket_r) 
# axs[0].set_thetagrids(ANG_MESH[:-1], [fr'${ANG_MESH[i]}^{{\circ}}$' if i%3 == 0 else None for i in range(len(ANG_MESH[:-1]))])
axs[0].set_thetagrids([0])
axs[0].set_rgrids(np.log(1 + ECC_MESH), ECC_MESH.round(1), color='b', fontsize=6)
axs[0].grid()
axs[0].set_title('L')
# fig.colorbar(pplot_1, ax=axs[0])

# sns.heatmap(dfv_l_d, cmap=sns.cm.rocket_r, ax=axs[1])
pplot_2 = axs[1].pcolormesh(ANG_MESH, np.log(1 + ECC_MESH), dfv_r_d_.to_numpy(), cmap=sns.cm.rocket_r) 
# axs[1].set_thetagrids(ANG_MESH[:-1], [fr'${ANG_MESH[i]}^{{\circ}}$' if i%3 == 0 else None for i in range(len(ANG_MESH[:-1]))])
axs[1].set_thetagrids([0])
axs[1].set_rgrids(np.log(1 + ECC_MESH), ECC_MESH.round(1), color='b', fontsize=6)
axs[1].grid()
axs[1].set_title('R')
# fig.colorbar(pplot_2, ax=axs[1])

fig.colorbar(pplot_2, cax=fig.add_axes([1, 0, 0.05, 0.95]))

fig.suptitle('Vertex (Voxel) Distance Heatmaps in Log Scale')
plt.tight_layout()

---

---

In [ ]:
# TODO:
## Aggregate on ang and plot on ecc

---

In [ ]:
# TODO:
## Distance calculations based on nearest centroid classification

In [ ]:
dfv1 = dfv[dfv[label].str.contains('V1')]
dfv2 = dfv[dfv[label].str.contains('V2')]

In [ ]:
v1 = dfv1[['x', 'y', 'z']].to_numpy()
v2 = dfv2[['x', 'y', 'z']].to_numpy()